In [1]:
import numpy as np
import librosa
import io
import soundfile as sf
from moviepy.editor import VideoFileClip
import pickle as pk
import os
import random
import tensorflow as tf
# from tensorflow.keras.saving import register_keras_serializable
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [2]:
real_audio_dir = (
    r"H:\.shortcut-targets-by-id\1jH_pc6mMj0Iu8wLS1r0vggMWpVElJvOU\SIH2024_DATASET\REAL"
)
fake_audio_dir = (
    r"H:\.shortcut-targets-by-id\1jH_pc6mMj0Iu8wLS1r0vggMWpVElJvOU\SIH2024_DATASET\FAKE"
)

In [3]:
def extract_features(file_path):
    try:
        # Load the video file
        video_clip = VideoFileClip(file_path)
        audio = video_clip.audio
        fps = audio.fps
        audio_samples = np.array(list(audio.iter_frames(fps=fps, dtype="float32"))).flatten()
        buffer = io.BytesIO()
        sf.write(buffer, audio_samples, fps, format='wav')
        buffer.seek(0)
        x, sr = librosa.load(buffer, sr=None)
        mfccs = librosa.feature.mfcc(y=x, sr=sr, n_mfcc=40)

        return mfccs

    except Exception as e:
        print(f"Error encountered while parsing file: {file_path}, {e}")
        return None


def load_data(real_dir, fake_dir):
    labels = []
    features = []

    # Load real audios
    for file_name in os.listdir(real_dir):
        file_path = os.path.join(real_dir, file_name)
        mfccs = extract_features(file_path)
        if mfccs is not None:
            features.append(mfccs)
            labels.append(0)  # 0 for REAL

    # Load fake audios
    for file_name in os.listdir(fake_dir):
        file_path = os.path.join(fake_dir, file_name)
        mfccs = extract_features(file_path)
        if mfccs is not None:
            features.append(mfccs)
            labels.append(1)  # 1 for FAKE

    return np.array(features), np.array(labels)

In [4]:
def extract_frame_features(file_path, frame_duration=1.0):
    try:
        video_clip = VideoFileClip(file_path)
        audio = video_clip.audio
        fps = audio.fps
        audio_samples = np.array(
            list(audio.iter_frames(fps=fps, dtype="float32"))
        ).flatten()
        buffer = io.BytesIO()
        sf.write(buffer, audio_samples, fps, format="wav")
        buffer.seek(0)
        x, sr = librosa.load(buffer, sr=None)

        # Split audio into frames of 'frame_duration' seconds
        frame_length = int(frame_duration * sr)
        frames = [
            librosa.feature.mfcc(y=x[i : i + frame_length], sr=sr, n_mfcc=40)
            for i in range(0, len(x), frame_length)
            if i + frame_length <= len(x)
        ]

        return frames  # Returns list of MFCCs for each frame

    except Exception as e:
        print(f"Error processing file {file_path}: {e}")
        return None

In [5]:
def load_data(real_dir, fake_dir):
    labels, features = [], []

    # Load real audio frames
    real_files = os.listdir(real_dir)
    for file_name in real_files:
        file_path = os.path.join(real_dir, file_name)
        frame_features = extract_frame_features(file_path)
        if frame_features:
            features.extend(frame_features)
            labels.extend([0] * len(frame_features))  # Label 0 for REAL

    # Limit fake audios to match the number of real samples
    fake_files = os.listdir(fake_dir)
    selected_fake_files = random.sample(
        fake_files, min(len(fake_files), len(real_files))
    )

    # Load fake audio frames
    for file_name in selected_fake_files:
        file_path = os.path.join(fake_dir, file_name)
        frame_features = extract_frame_features(file_path)
        if frame_features:
            features.extend(frame_features)
            labels.extend([1] * len(frame_features))  # Label 1 for FAKE

    return np.array(features), np.array(labels)

In [ ]:
X, y = load_data(real_audio_dir, fake_audio_dir)
X = X[..., np.newaxis]

In [ ]:
with open("X_for_dl.pkl", "wb") as f:
    pk.dump(X, f)
with open("y_for_dl.pkl", "wb") as f:
    pk.dump(y, f)

In [ ]:
# with open("X.pkl", "rb") as f:
#     X = pk.load(f)
# with open("y.pkl", "rb") as f:
#     y = pk.load(f)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [ ]:
# CNN-GRU with 1D Convolutions
model = models.Sequential(
    [
        # 1D Convolutional layers
        layers.Conv1D(
            64,
            kernel_size=5,
            strides=1,
            activation="relu",
            input_shape=(X.shape[1], X.shape[2]),
        ),
        layers.Conv1D(128, kernel_size=5, strides=1, activation="relu"),
        layers.MaxPooling1D(pool_size=2),
        layers.Dropout(0.3),
        # GRU layers
        layers.Bidirectional(layers.GRU(128, return_sequences=True)),
        layers.Bidirectional(layers.GRU(64)),
        # Fully connected layers
        layers.Dense(128, activation="relu"),
        layers.Dropout(0.5),
        layers.Dense(2, activation="softmax"),
    ]
)

In [ ]:
model.compile(
    optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

In [ ]:
checkpoint = ModelCheckpoint(
    "model/best_model.keras", monitor="val_loss", save_best_only=True
)
early_stopping = EarlyStopping(monitor="val_loss", patience=3)

In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 36, 64)         │        27,904 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 32, 128)        │        41,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 16, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 16, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 16, 256)        │       198,144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 128)            │       123,648 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 407,554 (1.55 MB)

 Trainable params: 407,554 (1.55 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model.fit(
    X_train,
    y_train,
    epochs=40,
    batch_size=16,
    validation_data=(X_test, y_test),
    callbacks=[checkpoint],
)

Epoch 1/40
148/148 ━━━━━━━━━━━━━━━━━━━━ 7s 14ms/step - accuracy: 0.5069 - loss: 0.7437 - val_accuracy: 0.5794 - val_loss: 0.6792
Epoch 2/40
148/148 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.5377 - loss: 0.6927 - val_accuracy: 0.5794 - val_loss: 0.6772
Epoch 3/40
148/148 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.5382 - loss: 0.6934 - val_accuracy: 0.5642 - val_loss: 0.6840
Epoch 4/40
128/148 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5219 - loss: 0.6926

KeyboardInterrupt: 

In [ ]:
# model = tf.keras.models.load_model("model/best_model.keras")
y_pred = model.predict(X_test)
y_pred_labels = np.argmax(y_pred, axis=1)

# Print classification report
print(classification_report(y_test, y_pred_labels, target_names=["REAL", "FAKE"]))

19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step
              precision    recall  f1-score   support

        REAL       0.59      0.48      0.53       303
        FAKE       0.54      0.65      0.59       289

    accuracy                           0.56       592
   macro avg       0.57      0.57      0.56       592
weighted avg       0.57      0.56      0.56       592



In [ ]:
def test_on_video(file_path, frame_duration=1.0):
    # Load the trained model
    model = tf.keras.models.load_model("model/best_model.keras")

    # Extract features for each frame in the new video
    frames = extract_frame_features(file_path, frame_duration)

    if frames is None:
        print("No frames extracted.")
        return

    # Reshape frames for model input
    frames = np.array(frames)[..., np.newaxis]

    # Predict on each frame
    predictions = model.predict(frames)
    pred_labels = np.argmax(predictions, axis=1)

    # Store deepfake frames
    deepfake_frames = []
    deepfake_indices = []

    # Identify deepfake frames
    for i, label in enumerate(pred_labels):
        if label == 1:  # If the label is FAKE
            deepfake_frames.append(frames[i])
            deepfake_indices.append(i)

    if not deepfake_frames:
        print("No deepfake frames detected in the video.")
        return

    # Analyze deepfake frames
    print(f"Found {len(deepfake_frames)} deepfake frames:")
    for i in deepfake_indices:
        status = "FAKE"
        print(f"Frame {i + 1}: {status}")

# Example usage
test_video_path = r"REAL\aelfnikyqj.mp4"  # Replace with your test video path
test_on_video(test_video_path)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 932ms/step
Found 1 deepfake frames:
Frame 10: FAKE


In [ ]:
# def test_on_video(file_path, frame_duration=1.0):
#     # Load the trained model
#     model = tf.keras.models.load_model("model/best_model.keras")

#     # Extract features for each frame in the new video
#     frames = extract_frame_features(file_path, frame_duration)

#     if frames is None:
#         print("No frames extracted.")
#         return

#     # Reshape frames for model input
#     frames = np.array(frames)[..., np.newaxis]

#     # Predict on each frame
#     predictions = model.predict(frames)
#     pred_labels = np.argmax(predictions, axis=1)

#     # Output results for each frame
#     for i, label in enumerate(pred_labels):
#         status = "REAL" if label == 0 else "FAKE"
#         print(f"Frame {i+1}: {status}")


# # Example usage
# test_video_path = r"REAL\bddjdhzfze.mp4"  # Replace with your test video path
# test_on_video(test_video_path)

In [ ]:
# @register_keras_serializable()
# class AudioModel(tf.keras.Model):
#     def __init__(self, input_shape):
#         super(AudioModel, self).__init__()
#         self.input_shape = input_shape  # Store the input shape
#         # Define the model layers
#         self.conv1 = layers.Conv2D(
#             32, kernel_size=(3, 3), activation="relu", input_shape=input_shape
#         )
#         self.conv2 = layers.Conv2D(64, kernel_size=(3, 3), activation="relu")
#         self.pool = layers.MaxPooling2D(pool_size=(2, 2))
#         self.dropout1 = layers.Dropout(0.25)

#         self.reshape = layers.Reshape((64, -1))
#         self.gru = layers.Bidirectional(layers.GRU(128, return_sequences=False))

#         self.dense1 = layers.Dense(128, activation="relu")
#         self.dropout2 = layers.Dropout(0.5)
#         self.dense2 = layers.Dense(2, activation="softmax")

#     def call(self, inputs):
#         # Forward pass through the layers
#         x = self.conv1(inputs)
#         x = self.conv2(x)
#         x = self.pool(x)
#         x = self.dropout1(x)

#         x = self.reshape(x)
#         x = self.gru(x)

#         x = self.dense1(x)
#         x = self.dropout2(x)
#         return self.dense2(x)

#     def get_config(self):
#         config = super(AudioModel, self).get_config()
#         config.update(
#             {"input_shape": self.input_shape}  # Include input shape in config
#         )
#         return config

#     @classmethod
#     def from_config(cls, config):
#         # Create a model instance from the config
#         input_shape = config.pop("input_shape")  # Extract input_shape from config
#         return cls(input_shape)  # Create an instance of the model


# # Function to create and compile the model
# def create_model(input_shape):
#     model = AudioModel(input_shape)
#     model.compile(
#         optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
#     )
#     return model


# # Example usage
# input_shape = (
#     64,
#     40,
#     1,
# )  # Adjust based on your data (e.g., (n_mfccs, time_steps, channels))

In [ ]:
# model = create_model(input_shape)
# model.summary()

In [ ]:
# checkpoint = ModelCheckpoint(r"models/dl_model.keras", monitor="val_loss", save_best_only=True, verbose=1)
# early_stopping = EarlyStopping(monitor="val_loss", patience=5, verbose=1)

# history = model.fit(
#     X_train, y_train, epochs=10, batch_size=16, validation_data=(X_test, y_test), callbacks=[checkpoint, early_stopping]
# )

In [ ]:
# model.save(r"models/dl_model.keras", overwrite=True)
# print("Model saved successfully.")

In [ ]:
# # Ensure to import keras properly
# import tensorflow as tf
# from tensorflow import keras


# # Function to load the model
# def load_model(model_path):
#     try:
#         # Load the model from the specified path
#         model = keras.models.load_model(model_path)
#         print("Model loaded successfully.")
#         return model
#     except Exception as e:
#         print(f"Error loading model: {e}")
#         return None

In [ ]:
# model_path = r"models/dl_model.keras"

# # Load the model
# loaded_model = load_model(model_path)